In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [47]:
v1=np.random.randint(50, 60, (5))
v2=np.random.randint(60, 70, (5))
v3=np.random.randint(50, 60, (6))
value=np.hstack([v1, v2, v3])
l1=np.zeros((5), int)
l2=np.ones((5), int)
l3=np.zeros((5), int)
label=np.hstack([l1, l2, l3])
d1=np.zeros((7), int)
d2=np.ones((8), int)
day=np.hstack([d1, d2])
data=pd.DataFrame({'value':value, 'label':[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]})
data

,value,label
0,51,0
1,58,0
2,54,1
3,51,0
4,58,0
5,63,0
6,62,0
7,69,1
8,60,0
9,67,0


In [48]:
def make_data(x, str,w):
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)

    return x

def make_label(x, w):
    window=w
    x['label_all']=x['label'].copy()
    for i in range(1, window):
        x.loc[(x['label_all']==1)|(x['label'].shift(i)==1), 'label_all']=1
        x.loc[(x['label_all']==0)&(x['label'].shift(i)==0), 'label_all']=0
    
    x['label']=x['label_all'].copy()
    return x[['label']]

In [49]:
window=3
df=make_data(data[['value']], 'value', window ).dropna()
display(data, df, df[::window])

,value,label
0,51,0
1,58,0
2,54,1
3,51,0
4,58,0
5,63,0
6,62,0
7,69,1
8,60,0
9,67,0


,value,value_lag1,value_lag2
2,54,58.0,51.0
3,51,54.0,58.0
4,58,51.0,54.0
5,63,58.0,51.0
6,62,63.0,58.0
7,69,62.0,63.0
8,60,69.0,62.0
9,67,60.0,69.0
10,52,67.0,60.0
11,51,52.0,67.0


,value,value_lag1,value_lag2
2,54,58.0,51.0
5,63,58.0,51.0
8,60,69.0,62.0
11,51,52.0,67.0
14,57,52.0,51.0


In [50]:
lag=[] 
window=3
lag.append(make_data(data[['value']], 'value', window))
lag.append(make_label(data[['label']], window))
df=pd.concat(lag, axis=1).dropna()
overlap=0
slide_step=window-overlap
step_df=df[::slide_step]
display(data, step_df)

,value,label
0,51,0
1,58,0
2,54,1
3,51,0
4,58,0
5,63,0
6,62,0
7,69,1
8,60,0
9,67,0


,value,value_lag1,value_lag2,label
2,54,58.0,51.0,1
5,63,58.0,51.0,0
8,60,69.0,62.0,1
11,51,52.0,67.0,1
14,57,52.0,51.0,1


In [51]:
anorm=step_df['label'].values
anorm

array([1, 0, 1, 1, 1], dtype=int64)

In [52]:
d=np.zeros(len(data))
for i in range(len(step_df)):
    d[i*slide_step:(i+1)*slide_step]=anorm[i]
d

array([1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [54]:
data['d']=d
data

,value,label,d
0,51,0,1.0
1,58,0,1.0
2,54,1,1.0
3,51,0,0.0
4,58,0,0.0
5,63,0,0.0
6,62,0,1.0
7,69,1,1.0
8,60,0,1.0
9,67,0,1.0


In [55]:
df_list=[]

lag=[] 
window=3
lag.append(make_data(data[['value']], 'value', window))
lag.append(make_label(data[ ['label']], window))
df_add_lag=pd.concat(lag, axis=1)
df_list.append(df_add_lag)

df_over=pd.concat(df_list).dropna()
display(data, df_over)

C:\Users\Arakawa\AppData\Local\Temp/ipykernel_7260/3267821568.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[column]=x[str].shift(i)
C:\Users\Arakawa\AppData\Local\Temp/ipykernel_7260/3267821568.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['label_all']=x['label'].copy()
C:\Users\Arakawa\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,value,label,d
0,51,0,1.0
1,58,0,1.0
2,54,1,1.0
3,51,0,0.0
4,58,0,0.0
5,63,0,0.0
6,62,0,1.0
7,69,1,1.0
8,60,0,1.0
9,67,0,1.0


,value,value_lag1,value_lag2,label
2,54,58.0,51.0,1
3,51,54.0,58.0,1
4,58,51.0,54.0,1
5,63,58.0,51.0,0
6,62,63.0,58.0,0
7,69,62.0,63.0,1
8,60,69.0,62.0,1
9,67,60.0,69.0,1
10,52,67.0,60.0,1
11,51,52.0,67.0,1


In [15]:
df=df.dropna().reset_index(drop=True)
display(df)

,value,value_lag1,value_lag2,label
0,53,56.0,54.0,0
1,50,53.0,56.0,0
2,57,50.0,53.0,0
3,61,57.0,50.0,1
4,68,61.0,57.0,1
5,68,65.0,60.0,1
6,58,68.0,65.0,1
7,51,58.0,68.0,1
8,58,51.0,58.0,0
9,55,58.0,51.0,0
